<a href="https://colab.research.google.com/github/omegaT4224/Andrew-Lee-Cruz/blob/main/AI_NFT_Validator_Blockchain_Build_V1_(Simulation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# AI NFT Validator Blockchain - Build V1 (Simulation)
# Implements the core logic based on the architectural design,
# simulating AI NFT validation and consensus.

import hashlib
import time
import random
import json
import pandas as pd
import io
from typing import List, Dict, Optional, Tuple, Any

# --- Configuration & Constants ---
CSV_FILE_PATH = 'Validation_Results.csv' # Assumes the file is accessible
CONSENSUS_THRESHOLD_PERCENT = 0.66 # e.g., 2/3 of validators needed for consensus
VALIDATOR_REWARD = 1.0 # Mock reward units per block validated
SLASH_AMOUNT = 10.0 # Mock stake slashed for failed/malicious validation
INITIAL_TOKEN_SUPPLY = 1_000_000 # Mock native token supply
STAKE_REQUIRED_TO_VALIDATE = 50.0 # Mock minimum stake required

# --- Helper Functions ---
def calculate_hash(index: int, previous_hash: str, timestamp: float, transactions: List[Dict], validator_info: Dict) -> str:
    """ Calculates the SHA-256 hash for a block's content. """
    block_content = {
        "index": index,
        "previous_hash": previous_hash,
        "timestamp": timestamp,
        "transactions": transactions,
        "validator_info": validator_info # Include validator ID(s) and potentially proof ref
    }
    # Use json.dumps with sort_keys for deterministic serialization
    block_string = json.dumps(block_content, sort_keys=True).encode()
    return hashlib.sha256(block_string).hexdigest()

# --- AI NFT Validator Class ---
class AINFTValidator:
    """ Represents an AI model tokenized as an NFT for validation. """
    def __init__(self, nft_id: str, owner_address: str, model_template_data: Dict):
        self.nft_id = nft_id
        self.owner_address = owner_address
        self.model_type = model_template_data.get('model_type', 'Simulated_AI_Template')
        self.model_template_id = model_template_data.get('template_id', 'N/A') # Link back to template source
        # Conceptual hash of the actual AI model's weights/logic (if available)
        self.model_hash = model_template_data.get('model_hash', hashlib.sha256(json.dumps(model_template_data, sort_keys=True).encode()).hexdigest())

        # Adapt performance metrics from the CSV data
        self.accuracy = min(max(model_template_data.get('Critical Temperature (T_c)', 0) / 50.0, 0.5), 0.99)
        self.speed_score = model_template_data.get('Critical Current Density (J_c)', 1000) / 10000
        self.robustness_score = model_template_data.get('Thermal Conductivity (kappa)', 1.0)
        self.complexity_score = model_template_data.get('Tensile Strength (sigma)', 100)

        self.is_registered_for_validation = False
        self.staked_amount = 0.0
        self.validation_history = [] # Track validation performance
        self.rewards_earned = 0.0

        print(f"Minted AI NFT {self.nft_id}: Owner={owner_address}, Type={self.model_type}, Accuracy={self.accuracy:.3f}")

    def register(self, stake_amount: float) -> bool:
        """ Register the NFT for validation if sufficient stake is provided. """
        if stake_amount >= STAKE_REQUIRED_TO_VALIDATE:
            self.staked_amount = stake_amount
            self.is_registered_for_validation = True
            print(f"NFT {self.nft_id} registered for validation with {stake_amount} stake.")
            return True
        else:
            print(f"NFT {self.nft_id} registration failed: Stake {stake_amount} is less than required {STAKE_REQUIRED_TO_VALIDATE}.")
            return False

    def unregister(self) -> float:
        """ Unregister the NFT and return the staked amount. """
        if not self.is_registered_for_validation:
            return 0.0
        unstaked = self.staked_amount
        self.staked_amount = 0.0
        self.is_registered_for_validation = False
        print(f"NFT {self.nft_id} unregistered. Returning {unstaked} stake.")
        return unstaked

    def simulate_off_chain_validation(self, transactions: List[Dict]) -> Tuple[bool, float, str]:
        """
        Simulates the OFF-CHAIN AI execution validating transactions.
        Returns: (overall_validity_decision, achieved_accuracy_metric, validation_details_log)
        """
        # --- This function represents the logic running in the off-chain module ---
        print(f"  [Off-Chain Sim] NFT {self.nft_id} validating {len(transactions)} transactions...")
        if not transactions:
            return True, 1.0, "No transactions to validate."

        correct_validations = 0
        inherently_invalid_tx_ids = [tx['id'] for tx in transactions if not tx.get('is_valid_simulation', True)]
        ai_flagged_invalid_tx_ids = []

        for tx in transactions:
            is_inherently_valid = tx.get('is_valid_simulation', True)
            # Simulate AI decision based on accuracy
            ai_thinks_valid = random.random() < self.accuracy
            is_correct = (ai_thinks_valid == is_inherently_valid)

            if is_correct:
                correct_validations += 1
            if not ai_thinks_valid: # AI flagged this transaction
                ai_flagged_invalid_tx_ids.append(tx['id'])

        achieved_accuracy = correct_validations / len(transactions)
        missed_invalid_txs = [tx_id for tx_id in inherently_invalid_tx_ids if tx_id not in ai_flagged_invalid_tx_ids]

        # Determine overall validity based on simulation
        validation_success = False
        details = ""
        if missed_invalid_txs:
            validation_success = False # Critical failure: missed an invalid transaction
            details = f"FAILED: Missed {len(missed_invalid_txs)} inherently invalid transaction(s) ({missed_invalid_txs}). Accuracy: {achieved_accuracy:.2f}."
            print(f"  [Off-Chain Sim] NFT {self.nft_id} Result: {details}")
        elif inherently_invalid_tx_ids:
             # Found all invalid transactions
             validation_success = True
             details = f"PASSED: Correctly identified all {len(inherently_invalid_tx_ids)} invalid transaction(s). Accuracy: {achieved_accuracy:.2f}."
             print(f"  [Off-Chain Sim] NFT {self.nft_id} Result: {details}")
        elif achieved_accuracy >= 0.85: # No invalid txs, accuracy high enough
            validation_success = True
            details = f"PASSED: No invalid transactions found. Accuracy sufficient ({achieved_accuracy:.2f})."
            print(f"  [Off-Chain Sim] NFT {self.nft_id} Result: {details}")
        else: # No invalid txs, but accuracy too low
            validation_success = False
            details = f"FAILED: No invalid transactions, but accuracy too low ({achieved_accuracy:.2f})."
            print(f"  [Off-Chain Sim] NFT {self.nft_id} Result: {details}")

        # Record history (could also happen after result submission on-chain)
        self.validation_history.append({
            "timestamp": time.time(), "tx_count": len(transactions),
            "result": validation_success, "accuracy_achieved": achieved_accuracy,
            "details": details
        })
        # --- End of Off-Chain Simulation ---

        # Return the result that would be submitted back to the chain
        return validation_success, achieved_accuracy, details # (e.g., signed by oracle/owner)

    def slash_stake(self) -> float:
        """ Reduces stake due to penalty. Returns amount slashed. """
        if self.staked_amount > 0:
            slashed = min(self.staked_amount, SLASH_AMOUNT)
            self.staked_amount -= slashed
            print(f"NFT {self.nft_id} slashed! Lost {slashed} stake. Remaining: {self.staked_amount}")
            # If stake falls below minimum, unregister
            if self.staked_amount < STAKE_REQUIRED_TO_VALIDATE:
                self.is_registered_for_validation = False
                print(f"NFT {self.nft_id} automatically unregistered due to low stake.")
            return slashed
        return 0.0

    def add_reward(self, amount: float):
        """ Adds rewards to the validator. """
        self.rewards_earned += amount

    def get_performance_summary(self) -> Dict:
        if not self.validation_history: return {"validations_run": 0}
        successful_runs = sum(1 for h in self.validation_history if h['result'])
        avg_accuracy = sum(h['accuracy_achieved'] for h in self.validation_history) / len(self.validation_history)
        return {
            "nft_id": self.nft_id, "owner": self.owner_address, "model_type": self.model_type,
            "base_accuracy": self.accuracy, "staked_amount": self.staked_amount,
            "rewards_earned": self.rewards_earned, "is_registered": self.is_registered_for_validation,
            "validations_run": len(self.validation_history), "successful_validations": successful_runs,
            "avg_achieved_accuracy": avg_accuracy
        }

    def __repr__(self):
        return f"AINFT(ID: {self.nft_id}, Owner: {self.owner_address}, Acc: {self.accuracy:.3f}, Stake: {self.staked_amount})"

# --- Blockchain Class ---
class Blockchain:
    """ Manages the chain, transactions, AI NFTs, and AI-driven consensus. """
    def __init__(self, validation_csv_path: str):
        self.chain: List[Dict] = []
        self.pending_transactions: List[Dict] = []
        self.ai_nft_validators: Dict[str, AINFTValidator] = {}
        self.nft_owners: Dict[str, List[str]] = {}
        self.nft_counter: int = 0
        self.native_token_supply: float = INITIAL_TOKEN_SUPPLY
        self.reward_pool: float = 0.0 # Funded by fees or inflation (simplified)
        self.balances: Dict[str, float] = {} # Track owner balances {address: balance}

        print("Initializing Blockchain...")
        self.ai_model_templates = self._load_ai_templates_from_csv(validation_csv_path)
        if not self.ai_model_templates:
             raise ValueError("Failed to load AI templates from CSV. Cannot initialize blockchain.")

        # Create the genesis block
        genesis_validator_info = {"validator_nft_id": "GENESIS", "consensus_details": "N/A"}
        genesis_block = self._create_block(index=0, previous_hash="0", validator_info=genesis_validator_info, transactions=[])
        self.chain.append(genesis_block)
        print("Blockchain Initialized with Genesis Block.")

    def _load_ai_templates_from_csv(self, csv_path: str) -> List[Dict]:
        templates = []
        try:
            # In this integrated environment, fetch content if available
            # For standalone: with open(csv_path, 'r') as f: csv_content_local = f.read()
            global csv_content # Use content fetched by the tool if available
            if not csv_content:
                 # Fallback: try to read the file directly (if running standalone)
                 print(f"CSV content not pre-fetched, attempting to read from {csv_path}")
                 with open(csv_path, 'r') as f:
                     csv_content = f.read()

            df = pd.read_csv(io.StringIO(csv_content))
            print(f"Loaded data from {csv_path} for AI NFT templates.")

            # Use 'Simulated' row (index 0)
            simulated_data_row = df.iloc[0]
            template_data = simulated_data_row.to_dict()
            template_data['template_id'] = 'Simulated_T1'
            template_data['model_type'] = 'Simulated_Material_AI_v1'
            templates.append(template_data)

            # Use 'Experimental' row if it exists (index 1)
            if len(df) > 1:
                 experimental_data_row = df.iloc[1]
                 exp_template = experimental_data_row.to_dict()
                 exp_template['template_id'] = 'Experimental_T2'
                 exp_template['model_type'] = 'Experimental_Material_AI_v1'
                 templates.append(exp_template)

            print(f"Created {len(templates)} AI NFT template(s) from CSV data.")
            return templates

        except FileNotFoundError:
            print(f"Error: CSV file not found at {csv_path}")
        except Exception as e:
            print(f"Error loading or processing CSV {csv_path}: {e}")

        # Fallback if loading fails
        print("Warning: Using default AI NFT template.")
        return [{
            'template_id': 'Default_F1', 'model_type': 'Default_Fallback_AI',
            'Critical Temperature (T_c)': 40.0, 'Critical Current Density (J_c)': 1000000.0,
            'Thermal Conductivity (kappa)': 2.0, 'Tensile Strength (sigma)': 150.0
        }]

    def mint_ai_nft(self, owner_address: str, template_index: int = 0) -> Optional[AINFTValidator]:
        if template_index >= len(self.ai_model_templates):
             print(f"Error: Template index {template_index} out of range.")
             return None

        self.nft_counter += 1
        nft_id = f"AI_NFT_{self.nft_counter}"
        template = self.ai_model_templates[template_index]

        new_nft = AINFTValidator(nft_id, owner_address, template)
        self.ai_nft_validators[nft_id] = new_nft
        self.nft_owners.setdefault(owner_address, []).append(nft_id)
        self.balances.setdefault(owner_address, 0.0) # Initialize balance if new owner
        return new_nft

    def register_validator(self, owner_address: str, nft_id: str, stake_amount: float) -> bool:
        """ Owner registers their NFT and stakes tokens. """
        if nft_id not in self.ai_nft_validators:
            print(f"Error: NFT {nft_id} does not exist.")
            return False
        nft = self.ai_nft_validators[nft_id]
        if nft.owner_address != owner_address:
            print(f"Error: Address {owner_address} does not own NFT {nft_id}.")
            return False
        if self.balances.get(owner_address, 0) < stake_amount:
             print(f"Error: Insufficient balance for {owner_address} to stake {stake_amount}.")
             return False

        if nft.register(stake_amount):
             # Deduct stake from owner's balance (simple model)
             self.balances[owner_address] -= stake_amount
             print(f"{owner_address} staked {stake_amount} for NFT {nft_id}. Remaining balance: {self.balances[owner_address]}")
             return True
        return False

    def unregister_validator(self, owner_address: str, nft_id: str):
        """ Owner unregisters their NFT and retrieves stake. """
        if nft_id not in self.ai_nft_validators or self.ai_nft_validators[nft_id].owner_address != owner_address:
             print(f"Error: Cannot unregister NFT {nft_id} for {owner_address}.")
             return
        nft = self.ai_nft_validators[nft_id]
        unstaked_amount = nft.unregister()
        # Add stake back to owner's balance
        self.balances[owner_address] += unstaked_amount
        print(f"{owner_address} unstaked {unstaked_amount} from NFT {nft_id}. New balance: {self.balances[owner_address]}")


    def add_transaction(self, sender: str, recipient: str, amount: float, fee: float = 0.01, is_valid_simulation: bool = True):
        """ Adds a new transaction to the pending pool, including a fee. """
        # Basic check for sender balance (simplified)
        if self.balances.get(sender, 0) < amount + fee:
             print(f"Transaction failed: Sender {sender} has insufficient balance for amount+fee.")
             return None

        tx_id = f"TX_{len(self.pending_transactions)}_{int(time.time()*1000)}"
        tx = {
            "id": tx_id, "sender": sender, "recipient": recipient, "amount": amount,
            "fee": fee, "timestamp": time.time(),
            "is_valid_simulation": is_valid_simulation
        }
        self.pending_transactions.append(tx)
        # Conceptually, fee could be moved to reward pool here or upon block inclusion
        self.reward_pool += fee
        # Deduct amount+fee from sender immediately for simplicity in this sim
        self.balances[sender] -= (amount + fee)
        print(f"Transaction {tx_id} added (Sender: {sender}, Amount: {amount}, Fee: {fee}, ValidSim: {is_valid_simulation}).")
        return len(self.chain) + 1 # Next block index

    def select_validator_nfts(self, num_validators: int = 3) -> List[AINFTValidator]:
        """ Selects multiple registered AI NFTs for consensus (stake-weighted random sampling). """
        registered_validators = [v for v in self.ai_nft_validators.values() if v.is_registered_for_validation and v.staked_amount > 0]
        if not registered_validators:
            return []

        stakes = np.array([v.staked_amount for v in registered_validators], dtype=float)
        probabilities = stakes / stakes.sum() if stakes.sum() > 0 else None

        # Ensure we don't select more validators than available
        num_to_select = min(num_validators, len(registered_validators))

        # Use numpy's choice for weighted sampling without replacement
        selected_nfts = list(np.random.choice(registered_validators, size=num_to_select, replace=False, p=probabilities))

        print(f"\nSelected {len(selected_nfts)} Validators (Stake-Weighted): {[v.nft_id for v in selected_nfts]}")
        return selected_nfts

    def _create_block(self, index: int, previous_hash: str, validator_info: Dict, transactions: List[Dict]) -> Dict:
        """ Creates a new block dictionary (internal helper). """
        timestamp = time.time()
        block = {
            'index': index,
            'timestamp': timestamp,
            'transactions': transactions,
            'previous_hash': previous_hash,
            'validator_info': validator_info, # e.g., {"consensus_validators": [id1, id2], "proof_ref": "..."}
            'hash': calculate_hash(index, previous_hash, timestamp, transactions, validator_info)
        }
        return block

    def attempt_block_creation(self, num_validators_for_consensus: int = 3) -> Optional[Dict]:
        """
        Orchestrates the AI NFT validation and consensus process to create a new block.
        """
        print("\n--- Attempting Block Creation ---")
        if not self.pending_transactions:
            print("No transactions pending.")
            return None

        # 1. Select multiple validators
        selected_validators = self.select_validator_nfts(num_validators=num_validators_for_consensus)
        if not selected_validators:
            print("Block Creation Failed: No registered validators available.")
            return None
        if len(selected_validators) < num_validators_for_consensus:
             print(f"Warning: Only selected {len(selected_validators)} validators (less than target {num_validators_for_consensus}). Adjusting threshold.")
             current_consensus_threshold = 1 if len(selected_validators) == 1 else CONSENSUS_THRESHOLD_PERCENT # Require 1 if only 1 selected

        else:
             current_consensus_threshold = CONSENSUS_THRESHOLD_PERCENT


        transactions_to_validate = list(self.pending_transactions) # Validate current pool

        # 2. Simulate Off-Chain Validation & Collect Results
        # (In reality, involves task assignment to oracles/owners and result submission)
        validation_results: Dict[str, Tuple[bool, float, str]] = {} # {nft_id: (result, accuracy, details)}
        print("Simulating Off-Chain Validation Task Assignment...")
        for validator in selected_validators:
            # Simulate the call to the off-chain module and getting the result back
            print(f"  Assigning validation task to {validator.nft_id}...")
            # The result tuple: (overall_validity_decision, achieved_accuracy_metric, validation_details_log)
            validation_results[validator.nft_id] = validator.simulate_off_chain_validation(transactions_to_validate)
            # Simulate result submission back to chain (placeholder)
            print(f"  Received validation result from {validator.nft_id}: {'PASSED' if validation_results[validator.nft_id][0] else 'FAILED'}")


        # 3. On-Chain Consensus Check
        print("\nPerforming On-Chain Consensus Check...")
        positive_votes = sum(1 for nft_id, result in validation_results.items() if result[0])
        total_votes = len(validation_results)
        agreement_ratio = positive_votes / total_votes if total_votes > 0 else 0

        print(f"  Consensus Results: {positive_votes}/{total_votes} voted PASSED (Agreement: {agreement_ratio:.2f})")
        consensus_reached = agreement_ratio >= current_consensus_threshold

        # --- Slashing/Rewarding Logic ---
        slashed_validators = []
        rewarded_validators = []
        if consensus_reached:
             print("  Consensus REACHED.")
             # Reward validators who voted correctly (PASSED)
             for nft_id, result in validation_results.items():
                 if result[0]: # Voted PASSED
                     self.ai_nft_validators[nft_id].add_reward(VALIDATOR_REWARD)
                     rewarded_validators.append(nft_id)
                 else: # Voted FAILED when consensus was PASSED - potential slash
                     slashed_amount = self.ai_nft_validators[nft_id].slash_stake()
                     self.reward_pool += slashed_amount # Add slashed stake to reward pool (example)
                     slashed_validators.append(nft_id)
        else:
             print("  Consensus FAILED.")
             # Reward validators who voted correctly (FAILED)
             for nft_id, result in validation_results.items():
                 if not result[0]: # Voted FAILED
                     self.ai_nft_validators[nft_id].add_reward(VALIDATOR_REWARD / 2) # Smaller reward for identifying failure?
                     rewarded_validators.append(nft_id)
                 else: # Voted PASSED when consensus was FAILED - potential slash
                     slashed_amount = self.ai_nft_validators[nft_id].slash_stake()
                     self.reward_pool += slashed_amount
                     slashed_validators.append(nft_id)

        # 4. Block Finalization (if consensus reached)
        if consensus_reached:
            print("Finalizing new block...")
            last_block = self.chain[-1]
            # Include info about the validators who reached consensus
            validator_info = {
                "consensus_validators": [nft_id for nft_id, result in validation_results.items() if result[0]],
                "all_participants": list(validation_results.keys()),
                "agreement_ratio": agreement_ratio,
                # "proof_reference": "placeholder_oracle_sig_or_zkp_ref" # Placeholder
            }
            new_block = self._create_block(
                index=len(self.chain),
                previous_hash=last_block['hash'],
                validator_info=validator_info,
                transactions=transactions_to_validate # Include the validated txs
            )
            self.chain.append(new_block)
            self.pending_transactions = [] # Clear the pool

            # Process transaction amounts (simple balance updates)
            for tx in new_block['transactions']:
                 self.balances.setdefault(tx['recipient'], 0.0)
                 self.balances[tx['recipient']] += tx['amount']
                 # Sender balance was already reduced when tx was added to pool

            print(f"Block #{new_block['index']} added to the chain.")
            print(f"  Rewarded Validators: {rewarded_validators}")
            print(f"  Slashed Validators: {slashed_validators}")
            return new_block
        else:
            print("Block not created due to failed consensus.")
            print(f"  Rewarded Validators (for voting FAILED): {rewarded_validators}")
            print(f"  Slashed Validators (for voting PASSED): {slashed_validators}")
            # Decide what to do with pending transactions (e.g., keep them, drop them)
            # self.pending_transactions = [] # Option: Drop transactions on failed consensus
            return None

    def is_chain_valid(self) -> bool:
        """ Checks blockchain integrity. """
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            # Recalculate hash and compare
            if current_block['hash'] != calculate_hash(
                current_block['index'], current_block['previous_hash'],
                current_block['timestamp'], current_block['transactions'],
                current_block['validator_info']
            ):
                print(f"Chain Error: Hash mismatch at block {current_block['index']}")
                return False
            # Check previous hash link
            if current_block['previous_hash'] != previous_block['hash']:
                print(f"Chain Error: Previous hash link broken at block {current_block['index']}")
                return False
        return True

    def print_balances(self):
        print("\n--- Account Balances ---")
        print(json.dumps(self.balances, indent=2))
        print("------------------------")

    def print_validator_summaries(self):
        print("\n--- AI NFT Validator Summaries ---")
        for nft_id, validator in self.ai_nft_validators.items():
            print(json.dumps(validator.get_performance_summary(), indent=2))
        print("--------------------------------")


# --- Main Execution Simulation ---
if __name__ == "__main__":
    print("--- Initializing AI NFT Validator Blockchain Build V1 ---")

    # Use a global variable to hold CSV content if fetched by the tool
    csv_content = None
    try:
        # Attempt to fetch content using the tool's mechanism if available
        # This mimics the environment where the file was uploaded
        print("Attempting to fetch CSV content...")
        csv_content = content_fetcher.fetch(query='validation results data', source_references=[ContentFetcher.SourceReference(id='uploaded:Validation_Results.csv')])
        print("CSV content fetched successfully.")
    except NameError:
        print("content_fetcher not available. Will try reading file directly.")
        csv_content = None # Ensure it's None if fetcher fails
    except Exception as e:
        print(f"Error fetching CSV content: {e}. Will try reading file directly.")
        csv_content = None


    # Initialize blockchain - uses fetched csv_content or reads file
    blockchain = Blockchain(CSV_FILE_PATH)

    # Setup initial balances for owners
    owner1 = "Alice_Addr"
    owner2 = "Bob_Addr"
    owner3 = "Charlie_Addr"
    blockchain.balances[owner1] = 1000.0
    blockchain.balances[owner2] = 1000.0
    blockchain.balances[owner3] = 1000.0

    # Mint AI NFTs
    print("\n--- Minting AI NFTs ---")
    nft1 = blockchain.mint_ai_nft(owner1, template_index=0) # Uses 'Simulated' data template
    nft2 = blockchain.mint_ai_nft(owner2, template_index=0)
    nft3 = blockchain.mint_ai_nft(owner1, template_index=1) # Uses 'Experimental' data template (if exists)
    nft4 = blockchain.mint_ai_nft(owner3, template_index=0)

    # Register and Stake Validators
    print("\n--- Registering & Staking Validators ---")
    if nft1: blockchain.register_validator(owner1, nft1.nft_id, 100.0)
    if nft2: blockchain.register_validator(owner2, nft2.nft_id, 150.0) # Higher stake
    if nft3: blockchain.register_validator(owner1, nft3.nft_id, 75.0)
    # nft4 is minted but not registered initially

    blockchain.print_balances()
    blockchain.print_validator_summaries()

    # Add Transactions
    print("\n--- Adding Transactions ---")
    blockchain.add_transaction(owner1, owner2, 50.0, fee=0.1, is_valid_simulation=True)
    blockchain.add_transaction(owner2, owner3, 25.0, fee=0.05, is_valid_simulation=True)
    blockchain.add_transaction(owner3, owner1, 10.0, fee=0.02, is_valid_simulation=False) # Invalid TX
    blockchain.add_transaction(owner1, owner3, 5.0, fee=0.01, is_valid_simulation=True)


    # Attempt Block Creation Round 1
    blockchain.attempt_block_creation(num_validators_for_consensus=3)

    blockchain.print_balances()

    # Add More Transactions
    print("\n--- Adding More Transactions ---")
    blockchain.add_transaction(owner2, owner1, 30.0, fee=0.05, is_valid_simulation=True)
    # Add many valid txs to test accuracy threshold without invalid ones
    for i in range(5):
         blockchain.add_transaction(owner3, f"Recipient_{i}", 1.0, fee=0.01, is_valid_simulation=True)


    # Attempt Block Creation Round 2
    blockchain.attempt_block_creation(num_validators_for_consensus=3)


    # --- Final State ---
    print("\n\n--- Blockchain Final State ---")
    print(f"Chain Length: {len(blockchain.chain)}")
    # print("Last 2 Blocks:")
    # for block in blockchain.chain[-2:]:
    #      print(json.dumps(block, indent=2))
    print(f"Pending Transactions: {len(blockchain.pending_transactions)}")
    print(f"Chain Integrity Valid: {blockchain.is_chain_valid()}")
    print(f"Total Reward Pool: {blockchain.reward_pool}")

    blockchain.print_balances()
    blockchain.print_validator_summaries()

--- Initializing AI NFT Validator Blockchain Build V1 ---
Attempting to fetch CSV content...
content_fetcher not available. Will try reading file directly.
Initializing Blockchain...
CSV content not pre-fetched, attempting to read from Validation_Results.csv
Error: CSV file not found at Validation_Results.csv
Blockchain Initialized with Genesis Block.

--- Minting AI NFTs ---
Minted AI NFT AI_NFT_1: Owner=Alice_Addr, Type=Default_Fallback_AI, Accuracy=0.800
Minted AI NFT AI_NFT_2: Owner=Bob_Addr, Type=Default_Fallback_AI, Accuracy=0.800
Error: Template index 1 out of range.
Minted AI NFT AI_NFT_3: Owner=Charlie_Addr, Type=Default_Fallback_AI, Accuracy=0.800

--- Registering & Staking Validators ---
NFT AI_NFT_1 registered for validation with 100.0 stake.
Alice_Addr staked 100.0 for NFT AI_NFT_1. Remaining balance: 900.0
NFT AI_NFT_2 registered for validation with 150.0 stake.
Bob_Addr staked 150.0 for NFT AI_NFT_2. Remaining balance: 850.0

--- Account Balances ---
{
  "Alice_Addr": 90

NameError: name 'np' is not defined